# Prioritized Experience Replay (PER)

**Prioritized Experience Replay (PER)** îmbunătățește replay buffer-ul clasic din DQN.  
În loc ca tranzițiile să fie selectate uniform, sampling-ul este ghidat de **prioritatea** fiecărei tranziții — cele mai „importante” sunt învățate mai des.

---

## 🎯 De ce avem nevoie de PER?

Unele tranziții din mediul RL sunt:

- **mult mai utile** (reward mare, evenimente rare)
- **mult mai informative** (TD-error mare → agentul greșește mult acolo)

Replay buffer-ul uniform tratează toate tranzițiile egal → învățare mai lentă.  
PER pune accent exact pe tranzițiile cu impact mare.

---

## 🔍 Principiul PER

Fiecărei tranziții i se atașează o prioritate:

$$
p_i = |TD\_\text{error, i}| + \epsilon
$$

Apoi probabilitatea de sampling devine:

$$
P(i) = \frac{p_i^\alpha}{\sum_j p_j^\alpha}
$$

unde **α** controlează cât de puternică este prioritizarea  
(α = 0 → sampling uniform, α = 1 → sampling proporțional total).

---

## 🧠 Importance Sampling Weights (IS)

Pentru a corecta bias-ul indus de sampling-ul neuniform, folosim:

$$
w_i = \left( \frac{1}{N \cdot P(i)} \right)^{\beta}
$$

unde **β** crește treptat către 1 în timp.  
Astfel, tranzițiile rare sau foarte prioritizate sunt echilibrate corect în loss.

---

## 🧱 Cum funcționează PER în practică?

1. Tranzițiile sunt salvate cu o prioritate inițială mare.  
2. Sampling-ul se face împărțind suma priorităților în segmente.  
3. Se extrag eșantioane proporțional cu prioritatea lor.  
4. Se calculează și se returnează:
   - tranzițiile  
   - indicii pentru update  
   - IS weights  
5. Prioritățile sunt actualizate după fiecare batch pe baza TD-error-ului nou.

---

## 📌 Avantajele PER

- Învață **mult mai rapid** decât sampling-ul uniform  
- Agentul se concentrează pe greșelile cele mai importante  
- Mai eficient în environment-uri complexe  
- Este componentă oficială din **Rainbow DQN**

---

## ⭐ Rezumat

**PER = Replay Buffer + Prioritizare după TD-error + Importance Sampling**  
→ agentul învață *mai inteligent*, nu aleator.

Este una dintre cele mai importante îmbunătățiri aduse lui DQN și baza multor algoritmi moderni.


In [ ]:
import gymnasium as gym
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tensorflow.keras import Model, layers, optimizers


# ============================================================
# SumTree (used by PER)
# ============================================================

class SumTree:
    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros(2 * capacity - 1, dtype=np.float32)
        self.data = np.zeros(capacity, dtype=object)
        self.write = 0

    def add(self, priority, data):
        idx = self.write + self.capacity - 1
        self.data[self.write] = data
        self.update(idx, priority)

        self.write += 1
        if self.write >= self.capacity:
            self.write = 0

    def update(self, idx, priority):
        change = priority - self.tree[idx]
        self.tree[idx] = priority

        # propagate upwards
        while idx != 0:
            idx = (idx - 1) // 2
            self.tree[idx] += change

    def get(self, s):
        idx = 0

        while True:
            left = 2 * idx + 1
            right = left + 1

            if left >= len(self.tree):
                break

            if s <= self.tree[left]:
                idx = left
            else:
                s -= self.tree[left]
                idx = right

        data_idx = idx - self.capacity + 1
        return idx, self.tree[idx], self.data[data_idx]

    @property
    def total(self):
        return self.tree[0]


# ============================================================
# Prioritized Replay Buffer (PER)
# ============================================================

class PERBuffer:
    def __init__(self, capacity, state_dim, alpha=0.6):
        self.tree = SumTree(capacity)
        self.capacity = capacity
        self.alpha = alpha
        self.max_priority = 1.0
        self.size = 0

        # vectorized storage
        self.state_dim = state_dim

    def store(self, state, next_state, action, reward, done):
        transition = (state, next_state, action, reward, done)

        # initial priority = max priority
        priority = self.max_priority ** self.alpha
        self.tree.add(priority, transition)

        self.size = min(self.size + 1, self.capacity)

    def sample(self, batch_size, beta=0.4):
        batch = []
        priorities = []
        idxs = []

        segment = self.tree.total / batch_size

        for i in range(batch_size):
            s = np.random.uniform(segment * i, segment * (i + 1))
            idx, p, data = self.tree.get(s)
            batch.append(data)
            priorities.append(p)
            idxs.append(idx)

        # unpack transitions
        states, next_states, actions, rewards, dones = map(
            np.array, zip(*batch)
        )

        # compute normalized importance sampling weights
        probs = np.array(priorities) / self.tree.total
        weights = (self.size * probs) ** (-beta)
        weights = weights / weights.max()

        return (
            states.astype(np.float32),
            next_states.astype(np.float32),
            actions.astype(np.int32),
            rewards.astype(np.float32),
            dones.astype(np.float32),
            np.array(idxs),
            weights.astype(np.float32),
        )

    def update_priorities(self, idxs, td_errors):
        td_errors = np.abs(td_errors) + 1e-5

        for idx, err in zip(idxs, td_errors):
            priority = err ** self.alpha
            self.tree.update(idx, priority)
            self.max_priority = max(self.max_priority, priority)


# ============================================================
# Dueling Network (same as before)
# ============================================================

class DuelingQNetwork(Model):
    def __init__(self, n_actions):
        super().__init__()

        self.fc1 = layers.Dense(128, activation='relu')
        self.fc2 = layers.Dense(128, activation='relu')

        self.value_fc = layers.Dense(128, activation='relu')
        self.adv_fc = layers.Dense(128, activation='relu')

        self.value = layers.Dense(1)
        self.advantage = layers.Dense(n_actions)

    def call(self, x):
        x = tf.convert_to_tensor(x, dtype=tf.float32)
        x = self.fc1(x)
        x = self.fc2(x)

        V = self.value(self.value_fc(x))
        A = self.advantage(self.adv_fc(x))

        A_mean = tf.reduce_mean(A, axis=1, keepdims=True)
        Q = V + (A - A_mean)

        return tf.clip_by_value(Q, -500, 500)

    def act(self, obs):
        obs = obs[np.newaxis, :].astype(np.float32)
        Q = self.call(obs)[0]
        return int(tf.argmax(Q).numpy())


# ============================================================
# Hyperparameters
# ============================================================

GAMMA = 0.99
LR = 3e-4
BATCH = 32
MEM_SIZE = 50000
NUM_EPISODES = 600

INITIAL_EPS = 1.0
FINAL_EPS = 0.02
EPS_DECAY = 0.995

TAU = 0.005
BETA_START = 0.4
BETA_INCREMENT = 1e-4


# ============================================================
# Soft Update
# ============================================================

def soft_update(target, online, tau=TAU):
    target_w = target.get_weights()
    online_w = online.get_weights()
    new_w = []

    for tw, ow in zip(target_w, online_w):
        new_w.append(tw * (1 - tau) + ow * tau)

    target.set_weights(new_w)


# ============================================================
# Training Loop (PER integrated)
# ============================================================

env = gym.make("CartPole-v1")
state_dim = env.observation_space.shape[0]
n_actions = env.action_space.n

buffer = PERBuffer(MEM_SIZE, state_dim)
online = DuelingQNetwork(n_actions)
target = DuelingQNetwork(n_actions)
target.set_weights(online.get_weights())

optimizer = optimizers.Adam(LR)
mse = tf.keras.losses.MeanSquaredError()

epsilon = INITIAL_EPS
beta = BETA_START
reward_history = []

for episode in range(NUM_EPISODES):

    state, _ = env.reset()
    ep_reward = 0

    for step in range(200):

        # ----------------------------
        # Epsilon-greedy
        # ----------------------------
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = online.act(state)

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        ep_reward += reward

        buffer.store(state, next_state, action, reward, float(done))
        state = next_state

        # ----------------------------
        # TRAINING
        # ----------------------------
        if buffer.size > 1000:

            states, next_states, actions, rewards, dones, idxs, weights = \
                buffer.sample(BATCH, beta)

            # Double DQN
            next_actions = tf.argmax(
                online(next_states), axis=1, output_type=tf.int32
            )
            indices = tf.stack(
                [tf.range(BATCH, dtype=tf.int32), next_actions], axis=1
            )

            next_Q = tf.gather_nd(target(next_states), indices)
            y = rewards + GAMMA * (1 - dones) * next_Q

            # ----------------------------
            # Compute loss (with weights)
            # ----------------------------
            with tf.GradientTape() as tape:
                q_pred = online(states)
                a_idx = tf.stack(
                    [tf.range(BATCH, dtype=tf.int32), actions], axis=1
                )
                pred = tf.gather_nd(q_pred, a_idx)

                td_errors = y - pred
                loss = tf.reduce_mean(weights * mse(pred, y))

            grads = tape.gradient(loss, online.trainable_variables)
            grads = [tf.clip_by_norm(g, 5.0) for g in grads]
            optimizer.apply_gradients(zip(grads, online.trainable_variables))

            # update priorities
            buffer.update_priorities(idxs, td_errors.numpy())

            # soft update
            soft_update(target, online)

        if done:
            break

    # decay epsilon & beta
    epsilon = max(FINAL_EPS, epsilon * EPS_DECAY)
    beta = min(1.0, beta + BETA_INCREMENT)

    reward_history.append(ep_reward)

    # ----------------------------
    # LIVE PLOT
    # ----------------------------
    if episode % 5 == 0:
        clear_output(wait=True)
        plt.figure(figsize=(10,4))

        # Reward
        plt.plot(
            reward_history,
            label="Reward",
            color="blue",
            alpha=0.3,   
            linewidth=1
        )

        # Moving average
        if len(reward_history) > 20:
            ma = np.convolve(reward_history, np.ones(20)/20, mode='valid')
            plt.plot(
                range(19, len(reward_history)),
                ma,
                label="Moving Avg (20 eps)",
                color="orange",
                linewidth=2.5
            )

        plt.title("PER DQN — Training Progress")
        plt.xlabel("Episode")
        plt.ylabel("Reward")
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()
